In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import gc
import json
import os
import random
from collections import Counter
from random import randint

import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances as ED
from tensorflow import keras
from tqdm import tqdm

In [4]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != "/device:GPU:0":
    raise SystemError("GPU device not found")
print("Found GPU at: {}".format(device_name))

Found GPU at: /device:GPU:0


In [5]:
tqdm.pandas()

In [6]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
RANDOM_STATE = 42
random.seed(RANDOM_STATE)
os.environ["PYTHONHASHSEED"] = str(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

In [8]:
morph_analyzer = Mystem()

In [9]:
DATASET_PATH = "/content/drive/My Drive/dataset"

Данные для обучения взяты из ноутбука RecSys notebook EDA в материалах курса.

In [10]:
interactions_df = pd.read_csv(f"{DATASET_PATH}/interactions_processed_kion.csv")
users_df = pd.read_csv(f"{DATASET_PATH}/users_processed_kion.csv")
items_df = pd.read_csv(f"{DATASET_PATH}/items_processed_kion.csv")

In [11]:
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72
1,699317,1659,2021-05-29,8317,100
2,656683,7107,2021-05-09,10,0
3,864613,7638,2021-07-05,14483,100
4,964868,9506,2021-04-30,6725,100


In [12]:
users_df.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,M,True
1,962099,age_18_24,income_20_40,M,False
2,1047345,age_45_54,income_40_60,F,False
3,721985,age_45_54,income_20_40,F,False
4,704055,age_35_44,income_60_90,F,False


In [13]:
items_df.rename(columns={"id": "item_id"}, inplace=True)

In [14]:
items_df.head()

,item_id,content_type,title,title_orig,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,release_year_cat
0,10711,film,поговори с ней,Hable con ella,"драмы, зарубежные, детективы, мелодрамы",испания,False,16.0,unknown,педро альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...",2000-2010
1,2508,film,голые перцы,Search Party,"зарубежные, приключения, комедии",сша,False,16.0,unknown,скот армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",2010-2020
2,10716,film,тактическая сила,Tactical Force,"криминал, зарубежные, триллеры, боевики, комедии",канада,False,16.0,unknown,адам п. калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...",2010-2020
3,7868,film,45 лет,45 Years,"драмы, зарубежные, мелодрамы",великобритания,False,16.0,unknown,эндрю хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...",2010-2020
4,16268,film,все решает мгновение,None,"драмы, спорт, советские, мелодрамы",ссср,False,12.0,ленфильм,виктор садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...",1970-1980


## Готовим фичи

### Фичи для юзеров

In [15]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом,
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()

,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,age_age_unknown,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,income_income_unknown,sex_F,sex_M,sex_sex_unknown,kids_flg_False,kids_flg_True
0,973171,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
1,962099,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
2,1047345,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0
3,721985,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
4,704055,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0


### Фичи для айтемов

In [16]:
item_cat_feats = ["content_type", "release_year_cat", "for_kids", "age_rating", "studios", "countries", "directors"]

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1)

items_ohe_df.head()

,item_id,content_type_film,content_type_series,release_year_cat_1920-1930,release_year_cat_1930-1940,release_year_cat_1940-1950,release_year_cat_1950-1960,release_year_cat_1960-1970,release_year_cat_1970-1980,release_year_cat_1980-1990,...,directors_ярив хоровиц,directors_ярон зильберман,directors_ярополк лапшин,directors_ярослав лупий,"directors_ярроу чейни, скотт моужер",directors_ясина сезар,directors_ясуоми умэцу,"directors_ёдзи фукуяма, ацуко фукусима, николас де креси, синъитиро ватанабэ, сёдзи кавамори",directors_ёлкин туйчиев,directors_ён сан-хо
0,10711,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2508,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10716,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7868,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16268,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### Сделаем матрицу взаимодействий

В ноутбуке с семинара было сказано, что в датасете есть непопулярные фильмы и неактивные пользователи. Можно их удалить. Также оставим только id юзеров и айтемов, которые есть во всех датасетах, остальные удалим

In [17]:
print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 10 процентов
interactions_df = interactions_df[interactions_df.watched_pct > 10]

# соберем всех пользователей, которые посмотрели
# больше 10 фильмов (можете выбрать другой порог)
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 962179
N items before: 15706

N users after: 79515
N items after: 6901


In [19]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_ohe_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_ohe_df.item_id.unique()))


print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

65974
6897


Соберем взаимодействия в матрицу user*item так, чтобы в строках этой матрицы были user_id, в столбцах - item_id, а на пересечениях строк и столбцов - единица, если пользователь взаимодействовал с айтемом и ноль, если нет.

Создадим некие внутренние индексы для user_id и item_id - uid и iid. Для этого просто соберем все user_id и item_id и пронумеруем их по порядку.

In [20]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
0,176549,9506,2021-05-11,4250,72,10616,3944
1,699317,1659,2021-05-29,8317,100,42131,675
6,1016458,354,2021-08-14,1672,25,61024,139
7,884009,693,2021-08-04,703,14,53150,279
14,5324,8437,2021-04-18,6598,92,310,3485


Отнормируем матрицу взаимодействий

In [21]:
interactions_vec = np.zeros((interactions_df.uid.nunique(), interactions_df.iid.nunique()))

# Используем информацию о качестве взаимодействия юзеров с айтемами для более репрезентативного сэмплирования
for user_id, item_id, weight in zip(interactions_df.uid, interactions_df.iid, interactions_df.watched_pct):
    interactions_vec[user_id, item_id] += weight / 100


res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

Соберем словари, которые хранят маппинг iid и uid в item_id и user_id и наоборот. Проиндексируем датасеты users_ohe_df и items_ohe_df по внутренним айди

In [22]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

In [23]:
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_ohe_df = items_ohe_df.set_index("iid")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid[x])
users_ohe_df = users_ohe_df.set_index("uid")

In [24]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims : n_dims * 2]
    negative = y_pred[:, n_dims * 2 : n_dims * 3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0)  # возвращаем ноль, если лосс отрицательный

    return loss

### Текстовые фичи

Давайте текстовые фичи преобразуем через tfidf. Но есть проблема - у нас текст обычный. Сделаем для него препроцессинг

In [25]:
items_df["description"].head()

0    Мелодрама легендарного Педро Альмодовара «Пого...
1    Уморительная современная комедия на популярную...
2    Профессиональный рестлер Стив Остин («Все или ...
3    Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...
4    Расчетливая чаровница из советского кинохита «...
Name: description, dtype: object

In [26]:
def convert_to_lowercase(text: str) -> str:
    return text.lower()


def delete_punctuation(text: str) -> str:
    return [word for word in wordpunct_tokenize(text) if word.isalpha() or word.isalnum()]


def delete_stop_words(words: str) -> str:
    return " ".join([word for word in words if word not in set(stopwords.words("russian"))])


def lemmatize(words: str):
    lemmatized_text = morph_analyzer.lemmatize(words)
    return [word for word in lemmatized_text if word.isalnum()]


def normalize(text: str) -> str:
    text = convert_to_lowercase(text)
    text = delete_punctuation(text)
    text = delete_stop_words(text)
    text = lemmatize(text)
    return text

In [27]:
gc.collect()

43

In [28]:
items_df["preprocess_description"] = items_df["description"].progress_apply(lambda x: normalize(x))

100%|██████████| 15963/15963 [04:05<00:00, 65.04it/s]


In [29]:
items_df["preprocess_description"].head(5)

0    [мелодрама, легендарный, педро, альмодовар, по...
1    [уморительный, современный, комедия, популярны...
2    [профессиональный, рестлер, стив, остин, темно...
3    [шарлотта, рэмплинг, кортни, джеральдин, джейм...
4    [расчетливый, чаровница, советский, кинохит, л...
Name: preprocess_description, dtype: object

In [30]:
items_df["preprocess_description"] = items_df["preprocess_description"].apply(lambda x: " ".join(x))

Добавим сюда векторизатор

In [31]:
tfidf_vectorizer = TfidfVectorizer(min_df=3, max_features=300)
vectorized = pd.DataFrame(
    tfidf_vectorizer.fit_transform(items_df["preprocess_description"]).toarray(),
    columns=tfidf_vectorizer.get_feature_names_out(),
)

In [32]:
vectorized.shape

(15963, 300)

In [33]:
vectorized["item_id"] = items_df["item_id"]
items_ohe_df = items_ohe_df.merge(vectorized, on="item_id", how="left")
items_ohe_df = items_ohe_df.fillna(0)

## Генератор и сэмплирование

In [34]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            # берем рандомный uid
            uid_i = randint(0, interactions.shape[0] - 1)
            # id хорошего айтема
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])
            # id плохого айтема
            neg_i = np.random.choice(range(interactions.shape[1]))
            # фичи юзера
            uid_meta.append(users.iloc[uid_i])
            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions_vec[uid_i])
            # фичи хорошего айтема
            pos.append(items.iloc[pos_i])
            # фичи плохого айтема
            neg.append(items.iloc[neg_i])

        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [
            np.array(uid_meta),
            np.array(uid_interaction),
        ]

In [35]:
# инициализируем генератор
gen = generator(
    items=items_ohe_df.drop(["item_id"], axis=1),
    users=users_ohe_df.drop(["user_id"], axis=1),
    interactions=interactions_vec,
)

ret = next(gen)


print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")

вектор фичей юзера: (1024, 19)
вектор взаимодействий юзера с айтемами: (1024, 6897)
вектор 'хорошего' айтема: (1024, 8888)
вектор 'плохого' айтема: (1024, 8888)

вектор фичей юзера: (1024, 19)
вектор взаимодействий юзера с айтемами: (1024, 6897)


In [36]:
N_FACTORS = 128

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1],)
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1],)

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1],)

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (8888,)
USER_META_MODEL_SHAPE: (19,)
USER_INTERACTION_MODEL_SHAPE: (6897,)


In [37]:
def item_model(n_factors=N_FACTORS):
    # входной слой
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

    # полносвязный слой
    layer_1 = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(inp)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2 = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(layer_1)

    add = keras.layers.Add()([layer_1, layer_2])

    # выходной слой
    out = keras.layers.Dense(
        N_FACTORS,
        activation="linear",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(add)

    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    # входной слой для вектора фичей юзера (из users_ohe_df)
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    # входной слой для вектора просмотров (из iteractions_vec)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

    # полносвязный слой
    layer_1_meta = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(inp_meta)

    layer_1_interaction = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(inp_interaction)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2_meta = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(layer_1_meta)

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])

    # конкатенируем вектор фичей с вектором просмотров
    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])

    # выходной слой
    out = keras.layers.Dense(
        N_FACTORS,
        activation="linear",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(concat_meta_interaction)

    return keras.models.Model([inp_meta, inp_interaction], out)


# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe_df)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
# вход для вектора просмотра юзера (из interactions_vec)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_interaction_in])
# получаем вектор "хорошего" айтема
pos = i2v(pos_in)
# получаем вектор "плохого" айтема
neg = i2v(neg_in)

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [38]:
model_name = "dssm"

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f"runs/{model_name}")

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor="loss", patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(filepath=model_name + "/epoch{epoch}-{loss:.2f}.h5", monitor="loss")

In [39]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

In [40]:
model.fit(
    generator(
        items=items_ohe_df.drop(["item_id"], axis=1),
        users=users_ohe_df.drop(["user_id"], axis=1),
        interactions=interactions_vec,
        batch_size=64,
    ),
    steps_per_epoch=100,
    epochs=30,
    initial_epoch=0,
    callbacks=[decay, t_board, check],
)

Epoch 1/30
100/100 [==============================] - ETA: 0s - loss: 0.3038

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 125s 1s/step - loss: 0.3038 - lr: 0.0010
Epoch 2/30
100/100 [==============================] - 118s 1s/step - loss: 0.2194 - lr: 0.0010
Epoch 3/30
100/100 [==============================] - 112s 1s/step - loss: 0.2006 - lr: 0.0010
Epoch 4/30
100/100 [==============================] - 112s 1s/step - loss: 0.1860 - lr: 0.0010
Epoch 5/30
100/100 [==============================] - 111s 1s/step - loss: 0.1706 - lr: 0.0010
Epoch 6/30
100/100 [==============================] - 107s 1s/step - loss: 0.1654 - lr: 0.0010
Epoch 7/30
100/100 [==============================] - 117s 1s/step - loss: 0.1652 - lr: 0.0010
Epoch 8/30
100/100 [==============================] - 111s 1s/step - loss: 0.1589 - lr: 0.0010
Epoch 9/30
100/100 [==============================] - 109s 1s/step - loss: 0.1542 - lr: 0.0010
Epoch 10/30
100/100 [==============================] - 109s 1s/step - loss: 0.1439 - lr: 0.0010
Epoch 11/30
100/100 [==============================] - 106s 

Пример предскзаания

In [41]:
# берем рандомного юзера
rand_uid = np.random.choice(list(users_ohe_df.index))

# получаем фичи юзера и вектор его просмотров айтемов
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid]
user_interaction_vec = interactions_vec[rand_uid]

# берем рандомный айтем
rand_iid = np.random.choice(list(items_ohe_df.index))
# получаем фичи айтема
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[rand_iid]

# получаем вектор юзера
user_vec = u2v.predict([np.array(user_meta_feats).reshape(1, -1), np.array(user_interaction_vec).reshape(1, -1)])

# и вектор айтема
item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))
ED(user_vec, item_vec)

1/1 [==============================] - 0s 56ms/step


array([[1.6428921]], dtype=float32)

Получим оффлайн рекомендации

In [42]:
items_features = items_ohe_df.drop(["item_id"], axis=1).to_numpy()
items_vecs = i2v.predict(items_features)

dists = ED(user_vec, items_vecs)

216/216 [==============================] - 1s 3ms/step


In [43]:
def get_reco(user_ids, items_vecs, num_reco=10):
    uids = [user_id_to_uid[user_id] for user_id in user_ids]
    user_features = users_ohe_df.drop(["user_id"], axis=1).iloc[uids]
    user_interaction_vec = interactions_vec[uids]

    user_vectors = u2v.predict([np.array(user_features), np.array(user_interaction_vec)])

    dists = ED(user_vectors, items_vecs)
    top10_iids = np.argsort(dists, axis=1)[:, :10]
    top10_iids_items = [iid_to_item_id[iid] for iid in top10_iids.reshape(-1)]
    top10_iids_items = np.array(top10_iids_items).reshape(top10_iids.shape)
    return top10_iids_items

In [44]:
users = list(user_id_to_uid.keys())

In [45]:
batch_size = 200
num_reco = 10
user_dict = {}
for i in range(0, len(users), batch_size):
    batch = users[i : i + batch_size]
    batch_recos = get_reco(batch, items_vecs)
    user_dict.update({int(user_id): [int(i) for i in reco.tolist()] for user_id, reco in zip(batch, batch_recos)})

6/6 [==============================] - 0s 3ms/step


In [47]:
with open("dssm.json", "w") as recos:
    json.dump(user_dict, recos)